In [ ]:
# from azureml.core import Workspace, Datastore
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
import torch
print(torch.cuda.is_available())

In [ ]:
# model_path = "codellama/CodeLlama-7b-Instruct-hf"
# model_path = "meta-llama/Llama-2-7b-chat-hf"
model_path = "./models/meta-llama/Llama-2-7b-chat-hf"
print(model_path)

In [ ]:
# Load the model in 4bit quantization for faster inference on smaller GPUs
conf = {
    "quantization_config": BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    ),
    "device_map": "auto",
}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

In [ ]:
# model.save_pretrained(f"./models/{model_path}")
tokenizer.save_pretrained(f"./models/{model_path}")

In [ ]:
model.save_pretrained(f"./models/{model_path}")

In [ ]:
# If all goes well, upload to blob storage:
workspace = Workspace.from_config()
ds = workspace.get_default_datastore()
ds.upload(f"./models/{model_path}", f"./base_models/{model_path}", show_progress=True, overwrite=True)

In [ ]:
model.push_to_hub("carlosgjs/Llama-2-7b-chat-hf-4bit")

In [ ]:
tokenizer.push_to_hub("carlosgjs/Llama-2-7b-chat-hf-4bit")